This is coded on an ESP8266 on the coffee level galvonometer
now hanging on the pipework at the front.

Very trivial code, no checking or async work

Also seems to fail connect to the broker 
on the esp8266 that it is wired to.  no idea

In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [2]:
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/umqtt.robust/umqtt/robust.py lib/umqtt/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/umqtt.simple/umqtt/simple.py lib/umqtt/
print(os.listdir("lib/umqtt"))


Sent 204 lines (6411 bytes) to lib/umqtt/simple.py.
['robust.py', 'simple.py']


In [31]:
%sendtofile main.py

import machine, time, network

wifiname = "DoESLiverpool-MQTT"
wifipassword = "decafbad00"
mqttbroker = "10.3.141.1"

# nodemcu8266 D1 = pin5, D2 = pin4; use pin13 on an esp32
p = machine.PWM(machine.Pin(5))  
p.duty(1023)
time.sleep(0.5)

sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)

p.duty(0)
sta_if.connect(wifiname, wifipassword)
while not sta_if.isconnected():
    time.sleep(0.1)
p.duty(1023)
print("connected to wifi")

from umqtt.robust import MQTTClient
client = MQTTClient("coffeedial", mqttbroker, 1883)
for i in range(100):
    try:
        client.connect()
        break
    except OSError as e:
        print(e)
    p.duty(1023*(i%2))
print("connected to mqtt")

def sub_cb(topic, msg):
    print((topic, msg))
    try:
        v = float(msg)
    except ValueError:
        print("ignore")
        return
    v = min(100, max(0, v))
    p.duty(int(v/100*1023))

client.set_callback(sub_cb)
client.subscribe(b"coffeedial/msg")
p.duty(500)

while True:
    client.wait_msg()


Sent 48 lines (1001 bytes) to main.py.


In [5]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [8]:
import os
print(os.listdir())

['boot.py', 'main.py', 'lib']


In [27]:
import network
si = network.WLAN()
si.active(True)

In [29]:
import esp
help(esp)

object <module 'esp'> is of type module
  __name__ -- esp
  osdebug -- <function>
  sleep_type -- <function>
  deepsleep -- <function>
  flash_id -- <function>
  flash_read -- <function>
  flash_write -- <function>
  flash_erase -- <function>
  flash_size -- <function>
  flash_user_start -- <function>
  neopixel_write -- <function>
  apa102_write -- <function>
  dht_readinto -- <function>
  freemem -- <function>
  meminfo -- <function>
  check_fw -- <function>
  info -- <function>
  malloc -- <function>
  free -- <function>
  esf_free_bufs -- <function>
  set_native_code_location -- <function>
  SLEEP_NONE -- 0
  SLEEP_LIGHT -- 1
  SLEEP_MODEM -- 2


In [28]:
print(si.scan())

scandone
[(b'mqtt-wifi', b"\xb8'\xeb\xf7\xb1%", 1, -49, 2, 0), (b'ProjectManifold', b'DNm\xd6\xb9\xd9', 1, -75, 3, 0), (b'DefProcEng', b'\xc0J\x00\x8d\x94r', 1, -82, 4, 0), (b'ProjectManifold guest access', b'FNm\xd6\xb9\xd9', 1, -70, 3, 0), (b'WiFi Guest', b'\xd88\xfc\x1c\xbd\xb8', 2, -72, 0, 0), (b'WiFi MAC', b'\xd88\xfc\\\xbd\xb8', 2, -72, 0, 0), (b'WiFi Secure', b'\xd88\xfc\x9c\xbd\xb8', 2, -72, 5, 0), (b'WiFi Guest', b'\xd88\xfc\x1cx\x08', 3, -70, 0, 0), (b'WiFi MAC', b'\xd88\xfc\\x\x08', 3, -71, 0, 0), (b'WiFi Secure', b'\xd88\xfc\x9cx\x08', 3, -70, 5, 0), (b'WiFi Guest', b'\xd88\xfc\x1c\xb5\xa8', 3, -75, 0, 0), (b'WiFi Secure', b'\xd88\xfc\x9c\xb5\xa8', 3, -75, 5, 0), (b'WiFi Guest', b'\xd88\xfc\x1c\xb5\x98', 5, -78, 0, 0), (b'WiFi MAC', b'\xd88\xfc\\\xb5\x98', 5, -80, 0, 0), (b'WiFi Secure', b'\xd88\xfc\x9c\xb5\x98', 5, -79, 5, 0), (b'WiFi MAC', b'\xd88\xfc\\\xab\x18', 6, -70, 0, 0), (b'WiFi Secure', b'\xd88\xfc\x9c\xab\x18', 6, -69, 5, 0), (b'WiFi Guest', b'\xd88\xfc\x1c\xab\x

In [24]:
import time
p.duty(0)
time.sleep(1)
p.duty(1000)
time.sleep_ms(60)
p.duty(0)
time.sleep_ms(20)
p.duty(500)
time.sleep(1)


.

In [8]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.9.4-8-ga9a3caad0 on 2018-05-11; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f5388052358, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


In [6]:
print(p)

PWM(5, freq=500, duty=1023)


In [158]:

import machine
p = machine.PWM(machine.Pin(5))
# nodemcu D1 = pin5, D2 = pin4


In [184]:
# this makes a very rapid move to the spot
# without the wobbling
# (could be implemented, and then use schedule to do the callback)
import time

p.duty(0)
time.sleep(1)

p.duty(1023)
time.sleep(0.09)
p.duty(0)
time.sleep(0.1)
p.duty(500)




.

In [235]:
%serialconnect

Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [236]:
import machine
a = machine.ADC(0)
p = machine.Pin(5, machine.Pin.OUT)


In [247]:
import time
p.value(1)
time.sleep_ms(50)
print(a.read())
p.value(0)
time.sleep_ms(50)
print(a.read())


212
9


In [234]:
p.value(0)
time.sleep_ms(50)
print(a.read())




***Connection broken [Input/output error]
You may need to reconnect
Closing serial Serial<id=0x7fa00504b588, open=True>(port='/dev/ttyUSB3', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands